# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Técnicas Avançadas para Captura e Tratamento de Dados</font>

## <font color="blue"> Web Scraping </font>

## <font color="blue">Solução dos Exercícios </font>

**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**

---

In [1]:
import requests as rq

### Exercício 1)
Considere a página do Wikipedia [https://en.wikipedia.org/robots.txt](https://en.wikipedia.org/robots.txt).
Faça uma requisição para pegar o conteúdo de tal página e:
1. verifique se a requisição foi bem sucedida
2. visualize o tipo de dado retornado e note que é um texto pleno (não é HTML nem JSON)
3. visualize o conteúdo

In [2]:
r = rq.get('https://en.wikipedia.org/robots.txt') # fazendo a requisição


print(r.status_code)  # verificando se a requisição foi bem sucedida

print(r.headers['Content-Type'])  # verificando o tipo de dado
                                  # retornado pelo request

print(r.text)  # verificando o conteúdo retornado

200
text/plain; charset=utf-8
﻿# robots.txt for http://www.wikipedia.org/ and friends
#
# Please note: There are a lot of pages on this site, and there are
# some misbehaved spiders out there that go _way_ too fast. If you're
# irresponsible, your access to the site may be blocked.
#

# Observed spamming large amounts of https://en.wikipedia.org/?curid=NNNNNN
# and ignoring 429 ratelimit responses, claims to respect robots:
# http://mj12bot.com/
User-agent: MJ12bot
Disallow: /

# advertising-related bots:
User-agent: Mediapartners-Google*
Disallow: /

# Wikipedia work bots:
User-agent: IsraBot
Disallow:

User-agent: Orthogaffe
Disallow:

# Crawlers that are kind enough to obey, but which we'd rather not have
# unless they're feeding search engines.
User-agent: UbiCrawler
Disallow: /

User-agent: DOC
Disallow: /

User-agent: Zao
Disallow: /

# Some bots are known to be trouble, particularly those designed to copy
# entire sites. Please obey robots.txt.
User-agent: sitecheck.internetseer

### Exercício 2)
Considere o site de dados abertos da cidade de São Paulo [http://dados.prefeitura.sp.gov.br/pt_PT/](http://dados.prefeitura.sp.gov.br/pt_PT/).

Faça uma requisição para pegar o conteúdo da página inicial e verifique se a requisição foi bem sucedida. Verifique que o tipo de dado retornado é um conteúdo HTML.

In [3]:
r = rq.get('http://dados.prefeitura.sp.gov.br/pt_PT/')

print(r.status_code)
print(r.headers)
print('\n\nTipo de conteudo:',r.headers['Content-Type'])

200
{'Date': 'Wed, 05 Jun 2024 22:29:33 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Connection': 'keep-alive', 'Cache-Control': 'private', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '7403'}


Tipo de conteudo: text/html; charset=utf-8


### Exercício 3)
Escreva um código para realizar um parsing no HTML obtido no exercício 2). Extraia o texto que descreve o portal e que está abaixo do título "SOBRE O PORTAL".

**DICA**: Utilize o browser para visualizar o "source" do html e encontre os marcadores que identificam o título "SOBRE O PORTAL".

In [4]:
from bs4 import BeautifulSoup

portal_soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
# a informação do portal esta dentro da marcação
# <div class="box">
about = portal_soup.find_all('div',{'class':'box'})

for a in about:
    p = a.find('p')
    if p:             # como existem várias marcações <div class="box">
                      # buscamos por aquela que contenha uma
                      # marcação interna tipo 'p' (paragrafo)
        print(p.text)


        O Portal de Dados Abertos da Prefeitura de São Paulo reúne conjuntos de dados de todas as secretarias, subprefeituras e empresas públicas municipais. Com os recursos deste Portal, é possível construir aplicativos, visualizações de dados e obter informações essenciais sobre as políticas públicas e a administração da cidade. 
		Saiba mais.



### Exercício 4)
Extraia todos os grupos de dados que estão disponíveis no portal [http://dados.prefeitura.sp.gov.br/pt_PT/](http://dados.prefeitura.sp.gov.br/pt_PT/), os quais estão indicados abaixo do título "CONHEÇA OS GRUPOS"). Imprima o nome do grupo e a URL da página do grupo.

**DICA**: Visualize o "source" do html e encontre os marcadores que identificam os grupos.

In [6]:
# a informação dos grupos esta dentro da marcação
# <li class="media-item home-media-item">
grupos = portal_soup('li',{'class':"media-item home-media-item"})

for g in grupos:
    name = g.find('h3').text
    url = g.find('a').get('href')
    print(name,'-->',url)

Cultura --> /pt_PT/group/cultura
Demografia --> /pt_PT/group/demografia
Direitos Humanos --> /pt_PT/group/direitos
Educação --> /pt_PT/group/educacao
Esporte e Lazer --> /pt_PT/group/esporte-e-lazer
Infraestrutura --> /pt_PT/group/infraestrutura
Meio Ambiente --> /pt_PT/group/meio-ambiente
Moradia --> /pt_PT/group/moradia
Negócios --> /pt_PT/group/negocios
Orçamento e Gestão --> /pt_PT/group/financas
Participação Social --> /pt_PT/group/participacao-social
Saúde e bem estar --> /pt_PT/group/saude
Segurança urbana --> /pt_PT/group/seguranca
Trabalho e renda --> /pt_PT/group/trabalho
Transportes e mobilidade --> /pt_PT/group/transporte


### Exercício 5)
A página ['http://dados.prefeitura.sp.gov.br/pt_PT/dataset'](http://dados.prefeitura.sp.gov.br/pt_PT/dataset) do portal possui uma barra de busca para que se realize consultas às bases existentes. O método <font color='blue'>get</font> pode realizar uma requisição onde parâmetros são enviandos para a barra de busca.
Por exemplo:
```python
rq.get('http://dados.prefeitura.sp.gov.br/pt_PT/dataset',
       params={'q','seguranca'})
```
envia a palavra 'seguranca' para a barra de busca, retornando um conteúdo HTML com o resultado da busca.

Faça uma requisição enviando como parâmetro de busca a palavra "saude". Faça um parsing no conteúdo retornado e extraia os nomes e as URLs dos repositórios retornados.
Por exemplo,

```html
Cadastro dos Estabelecimentos de Saúde

Este conjunto de dados contém informações sobre todos os equipamentos municipais de saúde, fruto de um esforço realizado pelo Departamento de Produção e Análise de Informação...
```
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="background-color:green">XLS</span>
<span style="background-color:orange">CSV</span>

corresponde a um dos repositórios listados no conteúdo retornado.
O nome do repositório é "Cadastro dos Estabelecimentos de Saúde" o qual está linkado à página [/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude](http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude).
Armaze o conteúdo extraído em um dicionário onde a chave é o nome do repositório e o valor é a URL correspondente.


In [2]:
import requests as rq
from bs4 import BeautifulSoup

In [3]:
url = "http://dados.prefeitura.sp.gov.br/pt_PT/dataset"
params = {"q": "saude"}
page = rq.get(
    url=url,
    params=params
)

In [4]:
page.status_code

200

In [5]:
pageb = BeautifulSoup(page.text, "html.parser")

In [7]:
lista = pageb.find_all("li", {"class": "dataset-item"})
count = 0
saude = {}

for l in lista:

  nome = l.find("h3").text
  url_ = l.find("ul", {"class": "dataset-resources"}).find("a").get("href")

  print(nome)
  print(url_)
  saude[nome] = url_


Cadastro dos Estabelecimentos de Saúde

/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude

Folha de Pagamento - HSPM

/pt_PT/dataset/folha-de-pagamento-hspm

Folha de Pagamento - AHMSP

/pt_PT/dataset/folha-de-pagamento-ahmsp

Pedidos de informação protocolados à Prefeitura via sistema e-SIC

/pt_PT/dataset/pedidos-de-informacao-protocolados-a-prefeitura-via-e-sic1

Relação de Servidores da Saúde do Município de São Paulo

/pt_PT/dataset/relacao-de-servidores-da-saude-do-municipio-de-sao-paulo

Índice de Desenvolvimento Humano Municipal (IDHM)

/pt_PT/dataset/indice-de-desenvolvimento-humano-municipal

Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de S...

/pt_PT/dataset/cadastro-nacional-de-enderecos-para-fins-estatisticos-censo-2010

Auditorias e Notas Técnicas - CGM

/pt_PT/dataset/auditorias-e-notas-tecnicas-cgm


: 

In [37]:
saude

{'\nCadastro dos Estabelecimentos de Saúde\n': '/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude',
 '\nFolha de Pagamento - HSPM\n': '/pt_PT/dataset/folha-de-pagamento-hspm',
 '\nFolha de Pagamento - AHMSP\n': '/pt_PT/dataset/folha-de-pagamento-ahmsp',
 '\nPedidos de informação protocolados à Prefeitura via sistema e-SIC\n': '/pt_PT/dataset/pedidos-de-informacao-protocolados-a-prefeitura-via-e-sic1',
 '\nRelação de Servidores da Saúde do Município de São Paulo\n': '/pt_PT/dataset/relacao-de-servidores-da-saude-do-municipio-de-sao-paulo',
 '\nÍndice de Desenvolvimento Humano Municipal (IDHM)\n': '/pt_PT/dataset/indice-de-desenvolvimento-humano-municipal',
 '\nCadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de S...\n': '/pt_PT/dataset/cadastro-nacional-de-enderecos-para-fins-estatisticos-censo-2010',
 '\nAuditorias e Notas Técnicas - CGM\n': '/pt_PT/dataset/auditorias-e-notas-tecnicas-cgm'}

### Exercício 6)
Cada uma das URLs obtidas no exercício 5) leva a um repositório contendo arquivos para download. Escreva um código para realizar uma requisição em todas as páginas apontadas pelas URLs obtidas no exercício 5). Armazene o resultado das requisições em uma lista chamada `repos`, ou seja, cada ítem da lista corresponde ao conteúdo retornado pela requisição feita a um repositório.

In [42]:
repos = []
url_ = "http://dados.prefeitura.sp.gov.br"

for s in saude.values():
  full_url = url_+s

  r = rq.get(full_url)
  if r.status_code == 200:
    print(full_url)
    repos.append(full_url)

print(len(repos))

http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-dos-estabelecimentos-de-saude
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/folha-de-pagamento-hspm
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/folha-de-pagamento-ahmsp
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/pedidos-de-informacao-protocolados-a-prefeitura-via-e-sic1
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/relacao-de-servidores-da-saude-do-municipio-de-sao-paulo
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/indice-de-desenvolvimento-humano-municipal
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/cadastro-nacional-de-enderecos-para-fins-estatisticos-censo-2010
http://dados.prefeitura.sp.gov.br/pt_PT/dataset/auditorias-e-notas-tecnicas-cgm
8


### Exercício 7)
Os conteúdos armazenados na lista `repos` são páginas HTML. Realize um parsing em cada um dos ítens da lista `repos` buscando pelo nome do repositório, a data de criação e a data da última atualização do repositório. Imprima tais informações.

In [51]:
rq.get(r).text

'\n<style>\n@font-face {\n    font-family: \'open_sansregular\';\n    src: url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.eot\');\n    src: url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.eot?#iefix\') format(\'embedded-opentype\'),\n         url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.woff2\') format(\'woff2\'),\n         url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.woff\') format(\'woff\'),\n         url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.ttf\') format(\'truetype\'),\n         url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-regular-webfont.svg#open_sansregular\') format(\'svg\');\n    font-weight: normal;\n    font-style: normal;\n\n}\n\n\n@font-face {\n    font-family: \'open_sansbold\';\n    src: url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-bold-webfont.eot\');\n    src: url(\'http://dados.prefeitura.sp.gov.br/fontes/opensans-bo

In [66]:
for r in repos:
    page = BeautifulSoup(rq.get(r).text, "html.parser")
    nome = page.find("h1").text.strip()

    section = page.find("section", {"class": "additional-info"})
    trs = page.find_all("tr")
    print(nome)

    for tr in trs:
      if "Última Atualização" in tr.find("th").text:
        atualizado = tr.find("td").text.strip()
        print("Última Atualização: ", atualizado)
      if "Data de criação" in tr.find("th").text:
        criacao = tr.find("td").text.strip()
        print("Data de criação: ", criacao)

    print()

Cadastro dos Estabelecimentos de Saúde
Última Atualização:  Dezembro 4, 2015, 22:12 (UTC)
Data de criação:  Dezembro 4, 2015, 21:50 (UTC)

Folha de Pagamento - HSPM
Última Atualização:  Maio 16, 2024, 12:23 (UTC)
Data de criação:  Dezembro 8, 2015, 18:54 (UTC)

Folha de Pagamento - AHMSP
Última Atualização:  Abril 5, 2022, 17:56 (UTC)
Data de criação:  Dezembro 8, 2015, 18:48 (UTC)

Pedidos de informação protocolados à Prefeitura via sistema e-SIC
Última Atualização:  Abril 6, 2023, 18:41 (UTC)
Data de criação:  Dezembro 4, 2015, 19:25 (UTC)

Relação de Servidores da Saúde do Município de São Paulo
Última Atualização:  Outubro 3, 2016, 16:25 (UTC)
Data de criação:  Outubro 3, 2016, 16:23 (UTC)

Índice de Desenvolvimento Humano Municipal (IDHM)
Última Atualização:  Dezembro 5, 2015, 15:58 (UTC)
Data de criação:  Dezembro 5, 2015, 15:37 (UTC)

Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de São Paulo
Última Atualização:  Dezembro 8, 2015, 19:41 (UTC)
Data de c

### Exercício 8)
Para cada repositório da lista `repos` imprima a quantidade total de arquivos XLS, CSV, etc., que estão disponíveis para download.

In [67]:
for r in repos:
    page = BeautifulSoup(rq.get(r).text, "html.parser")
    nome = page.find("h1").text.strip()
    items = page.find_all("li", {"class": "resource-item"})

    print(nome)
    print("qtd items: ", len(items))

Cadastro dos Estabelecimentos de Saúde
qtd items:  24
Folha de Pagamento - HSPM
qtd items:  16
Folha de Pagamento - AHMSP
qtd items:  13
Pedidos de informação protocolados à Prefeitura via sistema e-SIC
qtd items:  156
Relação de Servidores da Saúde do Município de São Paulo
qtd items:  2
Índice de Desenvolvimento Humano Municipal (IDHM)
qtd items:  4
Cadastro Nacional de Endereços para Fins Estatísticos, relativo à cidade de São Paulo
qtd items:  8
Auditorias e Notas Técnicas - CGM
qtd items:  390


### Exercício 9)
Alguns grupos de dados possuem muitos repositórios. Escreva um código para imprimir os nomes de todos os repositórios do grupo "Orçamento e Gestão".